In [1]:
import json
from pathlib import Path
import numpy as np


def read_jsonl(path):
    with open(path, 'r') as istream:
        return [json.loads(l) for l in istream]


DATA_FOLDER = Path.home() / "data/method_name_prediction/python/final/jsonl"

In [2]:
test = read_jsonl(DATA_FOLDER / "test_preprocessed.jsonl")

In [3]:
cubert_predictions = read_jsonl("cubert-generated.jsonl")

In [4]:
def compute_accuracy(predictions):
    top5_sum = 0
    top1_sum = 0
    for example in predictions:
        original = example['original']
        generated = example['generated']

        top_5 = 0
        for g in generated:
            cand, score = g['cand'], g['score']
            if cand == original:
                top_5 += 1
        top_5 %= 2
        top5_sum += top_5
        
        best_generated = max(generated, key=lambda e: e['score'])
        top_1 = int(best_generated['cand'] == original)
        top1_sum += top_1
    n_total = len(predictions)
    return {'acc1': top1_sum / n_total, 'acc5': top5_sum / n_total}

In [5]:
compute_accuracy(cubert_predictions)

{'acc1': 0.09027746034648261, 'acc5': 0.16464780362938244}

In [6]:
transformer_predictions = read_jsonl("transformer-generated.jsonl")

In [7]:
compute_accuracy(transformer_predictions)

{'acc1': 0.044933034693970836, 'acc5': 0.08227819170818668}

In [10]:
from rouge_metric import PerlRouge


rouge = PerlRouge(
    rouge_n_max=3,
    rouge_l=True,
    rouge_w=True,
    rouge_w_weight=1.2,
    rouge_s=True,
    rouge_su=True,
    skip_gap=4
)

In [20]:
def get_top_hypos(predictions):
    return [max(e['generated'], key=lambda c: c['score'])['cand'] for e in predictions]

def get_references(predictions):
    return [[e['original']] for e in predictions]

In [21]:
references = get_references(cubert_predictions)
cubert_top_hypos = get_top_hypos(cubert_predictions)

In [22]:
scores = rouge.evaluate(cubert_top_hypos, references)
print(scores)

CalledProcessError: Command '['perl', '/home/maxkvant/.local/lib/python3.7/site-packages/rouge_metric/RELEASE-1.5.5/ROUGE-1.5.5.pl', '-a', '-c', '95', '-e', '/home/maxkvant/.local/lib/python3.7/site-packages/rouge_metric/RELEASE-1.5.5/data', '-n', '3', '-r', '1000', '-2', '4', '-U', '-w', '1.2', '-f', 'A', '-p', '0.5', './.rouge_metric/tmp33_p59kl/config.xml']' returned non-zero exit status 2.

In [ ]:
references = get_references(transformer_predictions)
transformer_top_hypos = get_top_hypos(transformer_predictions)

In [ ]:
scores = rouge.evaluate(transformer_top_hypos, references)
print(scores)